Initialization  

Just importing necessary packages and connecting to the database. 

In [2]:
from telnetlib import STATUS
import psycopg2
import pandas as pd
import sys
from tqdm import tqdm

password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

username = 'piacobelli'
connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True


C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Row Count And Claim Count 

Creating an initial table to compare row counts, member ID counts, and claim ID counts between the raw data and the data warehouse. 

In [8]:
tableCreateQuery = ''' 
DROP TABLE IF EXISTS qa_reporting.dw_iqva_claim_diag_counts;
CREATE TABLE  qa_reporting.dw_iqva_claim_diag_counts
(
    calendar_year INT,
    dw_row_count BIGINT,
    src_row_count BIGINT,
    row_count_diff BIGINT,
    row_count_percentage FLOAT,
    dw_uth_clm_id_count BIGINT,
    dw_src_clm_id_count BIGINT,
    src_clm_count BIGINT,
    clm_count_diff BIGINT,
    clm_count_percentage FLOAT,
    dw_uth_mbr_id_count BIGINT,
    dw_src_mbr_id_count BIGINT,
    src_mbr_count BIGINT,
    mbr_count_diff BIGINT,
    mbr_count_percentage FLOAT,
    date_generated DATE
);
'''

with connection.cursor() as cursor:
    cursor.execute(tableCreateQuery)

In [5]:
with connection.cursor() as cursor:
    countInsertQuery = f'''
        INSERT INTO qa_reporting.dw_iqva_claim_diag_counts
        (
            calendar_year, 
            dw_row_count, 
            dw_uth_clm_id_count, 
            date_generated
        )
        SELECT 
            year, 
            COUNT(*), 
            COUNT(DISTINCT uth_claim_id), 
            current_date
        FROM dw_staging.iqva_claim_diag
        WHERE diag_position = '1'
        GROUP BY 1;
    '''
    
    cursor.execute(countInsertQuery)

    clmCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_diag_counts b
        SET dw_src_clm_id_count = count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT claim_id_src) AS count 
            FROM dw_staging.iqva_claim_diag
            WHERE diag_position = '1'
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year;
    '''

    cursor.execute(clmCountUpdateQuery)

    mbrCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_diag_counts b
        SET dw_uth_mbr_id_count = count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT uth_member_id) AS count 
            FROM dw_staging.iqva_claim_diag
            WHERE diag_position = '1'
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year;
    '''

    cursor.execute(mbrCountUpdateQuery)
    
    srcCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_diag_counts b
        SET dw_src_mbr_id_count = count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT member_id_src) AS count 
            FROM dw_staging.iqva_claim_diag
            WHERE diag_position = '1'
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year;
    '''

    cursor.execute(srcCountUpdateQuery)


In [3]:
with connection.cursor() as cursor: 

    compUpdateQuery = '''
        WITH clms AS (
            SELECT 
                min(year) as year, 
                pat_id, 
                derv_claimno
            FROM dev.pi_iqvia_derv_claimno_new_all_yr
            WHERE diag1 IS NOT NULL and (new_rectype != 'P' or new_rectype IS NULL)
            GROUP BY pat_id, derv_claimno
        ),
        clm_counts AS (
            SELECT 
                year, 
                COUNT(*) src_row_count, 
                COUNT(DISTINCT pat_id) src_mbr_count, 
                COUNT(DISTINCT derv_claimno) src_clm_count
            FROM clms
            GROUP BY 1
        )
        update qa_reporting.dw_iqva_claim_diag_counts a
        set src_row_count = b.src_row_count,
            row_count_diff = a.dw_row_count - b.src_row_count,
            row_count_percentage = 100. * abs(a.dw_row_count - b.src_row_count) / b.src_row_count,
            src_clm_count = b.src_clm_count,
            clm_count_diff = a.dw_uth_clm_id_count - b.src_clm_count,
            clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.src_clm_count) / b.src_clm_count,
            src_mbr_count = b.src_mbr_count,
            mbr_count_diff = a.dw_uth_mbr_id_count - b.src_mbr_count,   
            mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.src_mbr_count) / b.src_mbr_count
        from clm_counts b
        where a.calendar_year = b.year;
    '''

    cursor.execute(compUpdateQuery)

In [4]:

countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_diag_counts;', con=connection)
countsDf.sort_values(['clm_count_percentage'], ascending=False)
print(countsDf)

    calendar_year  dw_row_count  src_row_count  row_count_diff  \
0            2018     149488281      149488281               0   
1            2020     117453998      117453998               0   
2            2008     361159186      361159186               0   
3            2021     130141081      130141081               0   
4            2014     272912632      272912632               0   
5            2009     363615130      363615130               0   
6            2019     149581640      149581640               0   
7            2006     248842725      248842725               0   
8            2011     323010839      323010839               0   
9            2012     290772589      290772589               0   
10           2022     129199291      129199291               0   
11           2017     212469853      212469853               0   
12           2013     257613612      257613612               0   
13           2016     272592335      272592335               0   
14        

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_40252\1972483880.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_diag_counts;', con=connection)


Diagnosis Codes

Counting different diagnosis codes. 

In [7]:
dxCodeQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_diag_counts;
    SELECT 
        year, 
        diag_cd, 
        COUNT(*) AS diag_count
    INTO qa_reporting.dw_iqva_diag_counts
    FROM dw_staging.iqva_claim_diag
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(dxCodeQuery)

dxCodeDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_diag_counts;', con=connection)
print(dxCodeDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_38052\180123024.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dxCodeDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_diag_counts;', con=connection)


        year  diag_cd  diag_count
0       2018    Z6282           3
1       2020   M10421          10
2       2017    C9212         859
3       2007    73350          20
4       2009    36630        2669
...      ...      ...         ...
710780  2021     I447       97923
710781  2021  S90212A        2989
710782  2023  S36538D          11
710783  2017  S62658D          30
710784  2007     3339        1224

[710785 rows x 3 columns]


In [8]:
dxCodeAntiJoinQuery = '''
    SELECT a.*
    FROM qa_reporting.dw_iqva_diag_counts a
    LEFT JOIN reference_tables.ref_cms_icd_cm_codes b ON a.diag_cd = cd_value
    WHERE b.cd_value IS NULL;
'''

invalidDxDf = pd.read_sql(dxCodeAntiJoinQuery, con=connection)
invalidDxDf.groupby('year')['diag_count'].sum()
print(invalidDxDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_38052\2556203410.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalidDxDf = pd.read_sql(dxCodeAntiJoinQuery, con=connection)


       year  diag_cd  diag_count
0      2012     6497          41
1      2008     3705          27
2      2011     8197           3
3      2012    35100           5
4      2012    73919           1
...     ...      ...         ...
85492  2021   M25575           1
85493  2006     2754         326
85494  2007    84180           2
85495  2015   B353M2           1
85496  2017  R94R943           5

[85497 rows x 3 columns]


In [9]:
dxCompDf = pd.DataFrame(
    {
        'overall_diag_count': dxCodeDf.groupby('year')['diag_count'].sum(),
        'invalid_diag_count': invalidDxDf.groupby('year')['diag_count'].sum(),
        'valid_diag_count': dxCodeDf.groupby('year')['diag_count'].sum() - invalidDxDf.groupby('year')['diag_count'].sum()
    }
)
dxCompDf.loc[dxCompDf['valid_diag_count'].isna(),'valid_diag_count'] = dxCompDf.loc[dxCompDf['valid_diag_count'].isna(),'overall_diag_count']
dxCompDf['valid_diag_count'] =  dxCompDf['valid_diag_count'].astype(int)
dxCompDf['invalid_to_valid_percent'] = 100. * dxCompDf['invalid_diag_count'] / dxCompDf['valid_diag_count']
dxCompDf  

,overall_diag_count,invalid_diag_count,valid_diag_count,invalid_to_valid_percent
year,,,,
2006,440022096,3155055,436867041,0.722200
2007,591544902,3298443,588246459,0.560725
2008,685946048,3753204,682192844,0.550168
2009,702327116,3955141,698371975,0.566337
2010,629372189,2624207,626747982,0.418702
2011,673401912,4106834,669295078,0.613606
2012,613980598,2361664,611618934,0.386133
2013,560398839,1940626,558458213,0.347497
2014,606633795,4964118,601669677,0.825057


In [10]:
invalidDxDf.groupby('year').max()

,diag_cd,diag_count
year,,
2006,v5869,638525
2007,~,442090
2008,~,822500
2009,~,1182863
2010,~,467928
2011,~,568176
2012,ÎV0,291821
2013,¤,525311
2014,,2322908


ICD Version

Comparing counts for different ICD versions. 

In [12]:
icdVersQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_icd_version_count;

    SELECT 
        year, 
        icd_version, 
        COUNT(DISTINCT uth_claim_id) AS claim_count
    INTO qa_reporting.dw_iqva_icd_version_count
    FROM dw_staging.iqva_claim_diag
    GROUP BY 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(icdVersQuery)

In [14]:
icdVersDf = pd.read_sql('select * from qa_reporting.dw_iqva_icd_version_count where icd_version IS NOT NULL;', con=connection)
icdVersDf.sort_values(['year', 'icd_version'])

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_38052\718203148.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  icdVersDf = pd.read_sql('select * from qa_reporting.dw_iqva_icd_version_count where icd_version IS NOT NULL;', con=connection)


,year,icd_version,claim_count
25,2006,0,1
34,2006,9,237090969
13,2007,0,1
6,2007,9,305293184
28,2008,0,4
12,2008,9,341844339
32,2009,0,3
3,2009,9,344260819
20,2010,9,304219006
17,2011,0,2
